# Creating a project
In this notebook we'll use the `sc-api-tools` package to create a project on the platform, and show examples of how to interact with it

### Setting up the connection to the platform
First, we set up the connection to the cluster. This is done by instantiating a client, with the hostname (or ip address) and login details for the cluster. The login details are stored in the `.env` file and are loaded using the `dotenv` package, in the cell below.

In [1]:
from dotenv import dotenv_values

env_variables = dotenv_values(dotenv_path=".env")

if not env_variables:
    print("Unable to load login details from .env file, please make sure the file exists at the root of the notebooks directory.")

Now that the login details are loaded into the environment we can connect to the cluster

In [2]:
from sc_api_tools import SCRESTClient

client = SCRESTClient(
    host=env_variables.get('HOST'),
    username=env_variables.get('USERNAME'),
    password=env_variables.get('PASSWORD')
)

Authenticating on host https://sc-demo.iotg.sclab.intel.com/...
Authentication successful. Cookie received.


### Listing the projects currently in the workspace
To create or view projects on the platform, we have to set up a ProjectManager using the client we just created. Once the ProjectManager is set up, we can use it to print a list of projects that our workspace currently holds.

In [ ]:
from sc_api_tools.rest_managers import ProjectManager

project_manager = ProjectManager(session=client.session, workspace_id=client.workspace_id)

projects = project_manager.list_projects()

### Project creation parameters
To create a new project, we have to specify three things:
1. The name of the project
2. The type of the project
3. The label names or properties for each task in the project

##### Project name
The project name is easy, this can be any string you like, as long as there is no existing project with that name yet.

##### Project type
The project type requires more explanation, since this controls the tasks that will be added to the project's pipeline. 
- To create a single task project, simply pass the type of the task you want as the `project_type`. For example, to create a project with a single classification task, pass `project_type="classification"`
- In addition, we can construct arbitrary pipelines by passing a string that conforms to the format `{type_of_task_1}_to_{type_of_task_2}`. So for example, to create a project that holds a detection task followed by a segmentation task, we'd pass: `project_type="detection_to_segmentation"`. To understand which task types are supported, we can have a look at the TaskType class from `sc_api_tools`.

In [ ]:
from sc_api_tools.data_models.enums import TaskType

# Prints a list of the supported 'trainable' task types
print("Supported task types:")
for task_type in TaskType:
    if task_type.is_trainable:
        print("  " + str(task_type))

##### Labels
The `labels` parameter takes a nested list of label names, or a nested list of dictionaries representing label properties. For example, suppose we want to make a single task classification project with labels 'person', 'dog' and 'car'. In that case we should pass `labels=[['person', 'dog', 'car']]`. 

The list is nested because each entry in the outermost list corresponds to the labels for one of the tasks in the pipeline. For example, suppose we want to create a pipeline project of type `detection_to_classification`, with a label 'animal' for the detection task and labels 'dog', 'cat', 'horse', 'cow' for the classification task. In that case we'd pass `labels=[['animal'], ['dog', 'cat', 'horse', 'cow']]`. The first entry in the labels-list corresponds to the labels for the first task, the second entry to those for the second task in the pipeline and so on.

In case more complicated relationships between labels are required, we can specify the labels as dictionaries with certain properties instead of simple strings containing only their names. For example, to create a single task hierarchical classification project to classify 'animals' and 'vehicles' into subcategories, we could pass the following: 
```json
labels = [
    [
        {"name": "animal"}, 
        {"name": "dog", "parent_id": "animal"}, 
        {"name": "cat", "parent_id": "animal"}, 
        {"name": "vehicle"}, 
        {"name": "car", "parent_id": "vehicle"}, 
        {"name": "taxi", "parent_id": "vehicle"}, 
        {"name": "truck", "parent_id": "vehicle"}
    ]
]
``` 
It is also possible to make a multi-label classification task (meaning multiple labels can be assigned to a single image) by using the "group" keyword in the label property dictionary. Labels in different groups will be treated as independent (i.e. non-exclusive) from eachother. 


## Creating a simple project
Now that we understand the parameters, let's go ahead and create a new project:

In [ ]:
# First set the project parameters. Feel free to experiment here!
PROJECT_NAME = "Segmentation demo"
PROJECT_TYPE = "segmentation"
LABELS = [["dog", "cat", "horse"]]

In [ ]:
# Now, use the project manager to create the project
project = project_manager.create_project(
    project_name=PROJECT_NAME,
    project_type=PROJECT_TYPE, 
    labels=LABELS
)

## Interacting with the project
The `Project` object that is returned by the `project_manager.create_project()` method contains a representation of the project on the platform. There are several ways to interact with it. First of all, we can get a very brief `summary` of the project

In [ ]:
print(project.summary)

If we need to know more details, we can also look at the project `overview`

In [ ]:
print(project.overview)

Finally, the `project` object also supports several methods to quickly access some of its properties, for example to get a list of all trainable tasks in the project simply use `project.get_trainable_tasks()`. This will prove useful later on when we want to do more complicated things on the platform, such as triggering or monitoring a training job.

In [ ]:
task_list = project.get_trainable_tasks()
print(f"Project '{project.name}' contains {len(task_list)} trainable tasks.")
for task in task_list:
    print(task.summary)

The `project` object that was created by the `project_manager.create_project()` method can also be retrieved by calling `project_manager.get_project_by_name()`. This is useful if you don't want to create a new project, but would like to interact with an existing project instead

In [ ]:
project = project_manager.get_project_by_name(project_name=PROJECT_NAME)
print(project.summary)

## Creating a more complex project
Now that we created a simple project, let's try our hand at something more complex. In the cell below we'll set the parameters to create a detection -> classification project, with hierarchical labels for the classification task. The detection task will have the label `vehicle`, while the classification task will contain hierarchical labels to narrow down the precise vehicle category. 

In [ ]:
PIPELINE_PROJECT_NAME = "Detection to hierarchical classification demo"
PIPELINE_PROJECT_TYPE = "detection_to_classification"
PIPELINE_LABELS = [
                    ["vehicle"],
                    [ 
                        "car", 
                        {"name": "taxi", "parent_id": "car"}, 
                        {"name": "pick-up", "parent_id": "car"},
                        {"name": "sports car", "parent_id": "car"},
                        "truck",
                        "bus",
                        {"name": "van", "parent_id": "bus"},
                        {"name": "school bus", "parent_id": "bus"}
                    ]
                  ]

In [ ]:
pipeline_project = project_manager.create_project(
    project_name=PIPELINE_PROJECT_NAME,
    project_type=PIPELINE_PROJECT_TYPE, 
    labels=PIPELINE_LABELS
)

Let's look at the project summary again:

In [ ]:
print(pipeline_project.summary)

Note that the project summary doesn't include the information regarding label hierarchies. If we want to be sure that the hierarchical label structure has been set up correctly, we can use `project.overview` for a more detailed view of the project:

In [ ]:
print(pipeline_project.overview)

If you look carefully at the overview you'll note that the top-level classification labels (`car`, `truck` and `bus`) have been assigned the parent `vehicle`, from the detection task preceding the classification task. Furthermore, the `car` and `bus` classes have their subcategories assigned as we specified. 

Of course, you can also check the project in the UI for a more visual representation.

## Cleaning up
To clean up the workspace, let's delete the projects that we just created. The project manager provides a method for this `project_manager.delete_project`. You can pass either the name of the project or the `Project` object to it. Because deleting a project is not something to do lightly, the method will ask for confirmation before deleting the project. (Note, it is possible to skip the confirmation if you are sure you know what you're doing. Have a look at the method documentation if you want to find out how).

In [ ]:
# Delete the simple project
project_manager.delete_project(project)

In [ ]:
# Delete the pipeline project
project_manager.delete_project(pipeline_project)